In [1]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from functools import partial
geolocator = Nominatim(user_agent="Mi_aplicacion_intento1")
geocode = partial(geolocator.geocode, language="en")


In [2]:
df = pd.read_csv('Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv')
df.head()

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [3]:
df.shape

(27663, 18)

## Homogenización de variables salario_actual y compensaciones.

Vamos a comenzar por esta homogenización debido al alto grado de dificultad de la homogenización en las variables geográficas. 

Primero observamos la naturaleza de la variable

In [4]:
print("Valores", list(df["Annual salary"].sort_values()))

Valores ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '00', '1', '1,000', '1,050,000', '1,080,000', '1,100,000', '1,100,000', '1,100,000', '1,100,000', '1,150,000', '1,150,000', '1,150,000', '1,200,000', '1,200,000', '1,260,000', '1,300,000', '1,300,000', '1,324,000', '1,334,782', '1,360,000', '1,400,000', '1,442', '1,650,000', '1,900,000', '10,000', '10,000', '10,500', '10,500', '10,800', '100', '100', '100', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100,000', '100

In [5]:
df["Currency"].unique()

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'Other', 'CHF', 'ZAR',
       'SEK', 'HKD', 'JPY'], dtype=object)

Como se puede observar hay un valor de moneda que es otro. Esto puede "Ensuciar" mucho los datos, puesto que cualquier valor que asumamos puede generar un cambio muy grande en la muestra.

In [6]:
print("Tamaño muestra:", len(list(df[df["Currency"] == "Other"]["Annual salary"])), "Datos:" ,list(df[df["Currency"] != "Other"]["Annual salary"]))

Tamaño muestra: 145 Datos: ['55,000', '54,600', '34,000', '62,000', '60,000', '62,000', '33,000', '50,000', '112,000', '45,000', '47,500', '62,000', '100,000', '52,000', '32,000', '24,000', '85,000', '59,000', '98,000', '54,000', '74,000', '50,000', '63,000', '96,000', '44,500', '60,000', '62,000', '48,000', '140,000', '80,000', '39,000', '125,000', '230,000', '110,000', '50,000', '68,000', '41,000', '76,000', '78,000', '37,000', '100,000', '35,000', '65,000', '187,500', '110,000', '66,625', '144,600', '200,850', '120,000', '97,500', '96,000', '53,000', '95,000', '72,000', '52,000', '106,000', '115,000', '100,500', '33,200', '52,000', '52,000', '79,000', '104,000', '75,000', '52,000', '70,000', '84,000', '53,000', '34,000', '38,000', '87,000', '122,000', '45,000', '45,000', '88,000', '71,000', '71,000', '75,000', '80,000', '125,000', '65,000', '36,000', '55,000', '90,000', '105,000', '60,000', '65,000', '85,000', '39,520', '115,000', '76,000', '180,000', '90,000', '19,300', '65,000', '

Por otra parte, se observa que los datos que tienen currency en "other" son 145 y tienen dimensiones muy cambiantes. Para efectos de este ejercicio vamos a quitar esos datos. En la vida real, tocartía hacer un estudio al respecto, posiblemente algún método de autorrellenado, por ejemplo poner el promedio de otras condiciones en el salario funcionaria. 

In [7]:
df = df[df["Currency"] != "Other"]

Ahora, con el resto de la muestra observar y realizar lo siguiente:

Rápidamente, parece que los errores principales puenten ser: 
1. El uso de comas cuando hablamso de miles o millones.
2. algunos número con problemas de dimensiones, 1, 100, 1000. El principal problema en esto es saber la dimensión correta. Dado que es sueldo anual y que 12.000.000 COP es demasiado bajo (teniendo en cuenta que el salario minimo es más que eso en Colombia, asumimos que cualquier valor menor que 12.000.000 es ese valor *1000.

Por otra parte, asumimos lo siquiente de las tasas de cambio:
1. USD \$3,917.52
2. GBP \$5,434.23
3. CAD \$3,094.32
4. EUR \$4,471.69
5. AUD/NZD \$2,811.54 Asumimos que es AUD
6. CHF \$4,253.90
7. ZAR \$258.89
8. SEK \$422.01
9. HKD \$505.09
10. JPY \$34.13

In [8]:
TRM = {"USD":3917.52, "GBP": 5343.23, "CAD":3094.32, "EUR": 4471.69, "AUD/NZD": 2811.54,
       "CHF":4253.90, "ZAR":258.89, "SEK":422.01, "HKD":505.09, "JPY":34.13}

df["Other monetary comp"] = df["Other monetary comp"].fillna(0)

df.loc[:,"salario_anual"] = df["Annual salary"].apply(lambda x: re.sub('[^0-9]+', '', x))
df.loc[:,"salario_anual"] = pd.to_numeric(df["salario_anual"], downcast="float")

df.loc[:,"compensaciones"] = df["Other monetary comp"].apply(lambda x: re.sub('[^0-9]+', '', str(x)))
df.loc[:,"compensaciones"] = pd.to_numeric(df["compensaciones"], downcast="float")

df.loc[:,"salario_anual"] = [df.loc[i, "salario_anual"]*TRM[df.loc[i, "Currency"]] for i in df.index]
df.loc[:,"compensaciones"] = [df.loc[i, "compensaciones"]*TRM[df.loc[i, "Currency"]] for i in df.index]
df.loc[:,"salario_anual"] = df["salario_anual"].apply(lambda x: x*1000 if x < 12000000 else x)

In [9]:
df["Total_ingresos"]  = df.loc[:,"salario_anual"] + df.loc[:,"compensaciones"]

## Estandarización localización 

Intenté hacerlo por expresiones regulares pero la cantidad de ciudades y paises lo hacian muy dispendioso. Por lo tanto, utilicé un servicio que se llama Nominatim, es como google maps pero completamente gratis (y tiene un performace menos bueno). Inicialmente, cree una variable lugar que se compone de la concatenación de ciudad, estado y pais. De esta forma busqué cada uno de los "lugares".

In [10]:
df["Lugar"] = df["City"] + ", " + df["State"] + ", " + df["Country"]
df["Lugar"] = df["Lugar"].apply(lambda x: re.sub('[^A-Za-z, ]+', '', str(x).upper()))


In [11]:
lugares = {}
errores = []
contador = 0
for c in df["Lugar"].unique():
    contador += 1
    try:
        lugares[c] = geolocator.geocode(c.replace(".", ""), exactly_one=True,language="en", namedetails=True, addressdetails=True).raw["address"]
        print(contador, c, lugares[c])
    except:
        errores.append(c)

1 BOSTON, MASSACHUSETTS, UNITED STATES {'city': 'Boston', 'county': 'Suffolk County', 'state': 'Massachusetts', 'postcode': '02102', 'country': 'United States', 'country_code': 'us'}
2 NAN {'state': 'Nangarhar Province', 'country': 'Afghanistan', 'country_code': 'af'}
3 CHATTANOOGA, TENNESSEE, US {'city': 'Chattanooga', 'county': 'Hamilton County', 'state': 'Tennessee', 'postcode': '818', 'country': 'United States', 'country_code': 'us'}
4 MILWAUKEE, WISCONSIN, USA {'city': 'Milwaukee', 'county': 'Milwaukee County', 'state': 'Wisconsin', 'country': 'United States', 'country_code': 'us'}
5 GREENVILLE, SOUTH CAROLINA, US {'city': 'Greenville', 'county': 'Greenville County', 'state': 'South Carolina', 'country': 'United States', 'country_code': 'us'}
6 HANOVER, NEW HAMPSHIRE, USA {'town': 'Hanover', 'county': 'Grafton County', 'state': 'New Hampshire', 'country': 'United States', 'country_code': 'us'}
7 COLUMBIA, SOUTH CAROLINA, USA {'city': 'Columbia', 'county': 'Richland County', 'state

53 DISTRICT OF COLUMBIA, DISTRICT OF COLUMBIA, UNITED STATES {'amenity': 'University of California Washington Center', 'house_number': '1608', 'road': 'Rhode Island Avenue Northwest', 'city': 'Washington', 'state': 'District of Columbia', 'postcode': '20036', 'country': 'United States', 'country_code': 'us'}
54 BLOOMINGTON, ILLINOIS, UNITED STATES {'city': 'Bloomington', 'county': 'McLean County', 'state': 'Illinois', 'country': 'United States', 'country_code': 'us'}
55 DENVER, COLORADO, US {'city': 'Denver', 'state': 'Colorado', 'country': 'United States', 'country_code': 'us'}
56 NEW YORK CITY, NEW YORK, UNITED STATES {'city': 'New York', 'state': 'New York', 'country': 'United States', 'country_code': 'us'}
57 COLUMBIA, SOUTH CAROLINA, UNITED STATES {'city': 'Columbia', 'county': 'Richland County', 'state': 'South Carolina', 'country': 'United States', 'country_code': 'us'}
60 CHAMPAIGN, ILLINOIS, USA {'city': 'Champaign', 'county': 'Champaign County', 'state': 'Illinois', 'country'

107 SAN ANTONIO, TEXAS, US {'city': 'San Antonio', 'county': 'Bexar County', 'state': 'Texas', 'postcode': '78205', 'country': 'United States', 'country_code': 'us'}
108 MCLEAN, VIRGINIA, UNITED STATES {'locality': 'McLean', 'hamlet': 'Brookhaven', 'county': 'Fairfax County', 'state': 'Virginia', 'postcode': '22101', 'country': 'United States', 'country_code': 'us'}
109 ARDMORE, OKLAHOMA, UNITED STATES {'town': 'Ardmore', 'county': 'Carter County', 'state': 'Oklahoma', 'country': 'United States', 'country_code': 'us'}
110 COLUMBUS, OHIO, US {'city': 'Columbus', 'county': 'Franklin County', 'state': 'Ohio', 'country': 'United States', 'country_code': 'us'}
111 SHAKER HTS, OHIO, US {'town': 'Shaker Heights', 'county': 'Cuyahoga County', 'state': 'Ohio', 'postcode': '44120', 'country': 'United States', 'country_code': 'us'}
112 CHARLESTON, SOUTH CAROLINA, USA {'city': 'Charleston', 'county': 'Charleston County', 'state': 'South Carolina', 'country': 'United States', 'country_code': 'us'}


161 STAMFORD, CONNECTICUT, UNITED STATES {'city': 'Stamford', 'county': 'Fairfield County', 'state': 'Connecticut', 'country': 'United States', 'country_code': 'us'}
162 NEWPORT, OREGON, UNITED STATES {'town': 'Newport', 'county': 'Lincoln County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
163 LOS ANGELES, CALIFORNIA, UNITED STATES OF AMERICA {'city': 'Los Angeles', 'county': 'Los Angeles County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
164 NEW BRUNSWICK, NEW JERSEY, UNITED STATES {'city': 'New Brunswick', 'county': 'Middlesex County', 'state': 'New Jersey', 'postcode': '08901', 'country': 'United States', 'country_code': 'us'}
166 SPOKANE, WASHINGTON, USA {'city': 'Spokane', 'county': 'Spokane County', 'state': 'Washington', 'country': 'United States', 'country_code': 'us'}
167 DURHAM, NORTH CAROLINA, US {'city': 'Durham', 'county': 'Durham County', 'state': 'North Carolina', 'country': 'United States', 'country_code': 'us'}


214 FAIRFIELD, CONNECTICUT, UNITED STATES {'town': 'Fairfield', 'county': 'Fairfield County', 'state': 'Connecticut', 'country': 'United States', 'country_code': 'us'}
215 MANCHESTER, CONNECTICUT, US {'boundary': 'Central Manchester', 'town': 'Vernon', 'county': 'Tolland County', 'state': 'Connecticut', 'country': 'United States', 'country_code': 'us'}
216 PITTSBURGH, PENNSYLVANIA, US {'city': 'Pittsburgh', 'county': 'Allegheny County', 'state': 'Pennsylvania', 'country': 'United States', 'country_code': 'us'}
217 DFW, TEXAS, USA {'aeroway': 'Dallas/Fort Worth International Airport', 'city': 'Grapevine', 'county': 'Tarrant County', 'state': 'Texas', 'postcode': '75261', 'country': 'United States', 'country_code': 'us'}
218 DC, DISTRICT OF COLUMBIA, USA {'city': 'Washington', 'state': 'District of Columbia', 'country': 'United States', 'country_code': 'us'}
219 WILMINGTON, NORTH CAROLINA, US {'city': 'Wilmington', 'county': 'New Hanover County', 'state': 'North Carolina', 'country': 'Un

...

6032 MADISON HEIGHTS, MICHIGAN, USA {'town': 'Madison Heights', 'county': 'Oakland County', 'state': 'Michigan', 'postcode': '48071', 'country': 'United States', 'country_code': 'us'}
6033 HAZLET, NEW JERSEY, US {'town': 'Hazlet', 'township': 'Hazlet Township', 'county': 'Monmouth County', 'state': 'New Jersey', 'postcode': '07730', 'country': 'United States', 'country_code': 'us'}
6034 MADISON, MISSISSIPPI, US {'town': 'Madison', 'county': 'Madison County', 'state': 'Mississippi', 'country': 'United States', 'country_code': 'us'}
6035 MARYVILLE , ILLINOIS, UNITED STATES {'village': 'Maryville', 'county': 'Madison County', 'state': 'Illinois', 'postcode': '62062', 'country': 'United States', 'country_code': 'us'}
6036 MORENCI, ARIZONA, USA {'town': 'Morenci', 'county': 'Greenlee County', 'state': 'Arizona', 'country': 'United States', 'country_code': 'us'}
6037 MOON TWP PA, PENNSYLVANIA, UNITED STATES {'town': 'Moon Township', 'county': 'Allegheny County', 'state': 'Pennsylvania', 'cou

6089 CHICAGO, ILLINOIS, ISA {'road': 'Isa Avenue', 'town': 'Chicago Heights', 'municipality': 'Bloom Township', 'county': 'Cook County', 'state': 'Illinois', 'postcode': '60411', 'country': 'United States', 'country_code': 'us'}
6090 RICHARDSON, TEXAS, USA  {'city': 'Richardson', 'county': 'Dallas County', 'state': 'Texas', 'country': 'United States', 'country_code': 'us'}
6091 LONG BEACH, NEW YORK, USA {'town': 'Long Beach', 'city': 'Long Beach', 'county': 'Nassau County', 'state': 'New York', 'postcode': '11561', 'country': 'United States', 'country_code': 'us'}
6092 MIAMI , FLORIDA, USA {'city': 'Miami', 'county': 'Miami-Dade County', 'state': 'Florida', 'country': 'United States', 'country_code': 'us'}
6093 NOLA, LOUISIANA, USA {'city': 'New Orleans', 'county': 'Orleans Parish', 'state': 'Louisiana', 'postcode': '70116:70119', 'country': 'United States', 'country_code': 'us'}
6094 CHARLTON, MASSACHUSETTS, USA {'town': 'Charlton', 'county': 'Worcester County', 'state': 'Massachusett

6140 CERES, CALIFORNIA, UNITED STATES {'town': 'Ceres', 'county': 'Stanislaus County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
6141 BOUNTIFUL, UTAH, US {'town': 'Bountiful', 'county': 'Davis County', 'state': 'Utah', 'postcode': '84010', 'country': 'United States', 'country_code': 'us'}
6142 REMOTE, MISSOURI, UNITED STATES {'highway': 'Remote Control Car Track', 'road': 'Parkside Drive', 'city': 'Columbia', 'county': 'Boone County', 'state': 'Missouri', 'postcode': '65202', 'country': 'United States', 'country_code': 'us'}
6144 MURRAY, UTAH, UNITED STATES {'town': 'Murray', 'city': 'Murray', 'county': 'Salt Lake County', 'state': 'Utah', 'postcode': '84107', 'country': 'United States', 'country_code': 'us'}
6145 SADDLEBROOK, NEW JERSEY, USA {'road': 'Saddlebrook Drive', 'township': 'Evesham Township', 'county': 'Burlington County', 'state': 'New Jersey', 'postcode': '08053', 'country': 'United States', 'country_code': 'us'}
6146 STOWE, VERMONT, USA {'to

6193 COLUMBUS , OHIO, USA  {'city': 'Columbus', 'county': 'Franklin County', 'state': 'Ohio', 'country': 'United States', 'country_code': 'us'}
6194 KENNEDY SPACE CENTER, FLORIDA, USA {'aeroway': 'John F. Kennedy Space Center', 'county': 'Brevard County', 'state': 'Florida', 'postcode': '32899', 'country': 'United States', 'country_code': 'us'}
6195 PORTLAND , OREGON, UNITED STATES OF AMERICA  {'city': 'Portland', 'county': 'Multnomah County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
6196 STERLING HEIGHTS, MICHIGAN, USA {'city': 'Sterling Heights', 'town': 'Sterling Heights', 'county': 'Macomb County', 'state': 'Michigan', 'postcode': '48313-4615', 'country': 'United States', 'country_code': 'us'}
6197 REMOTE, ILLINOIS, USA  {'building': 'Remote Parking ATS Station (Closed)', 'road': 'Coffey Road', 'suburb': "O'Hare", 'city': 'Chicago', 'municipality': 'Jefferson Township', 'county': 'Cook County', 'state': 'Illinois', 'postcode': '60666', 'country': 'United

6249 PALATINE, ILLINOIS, UNITED STATES {'town': 'Palatine', 'municipality': 'Palatine Township', 'county': 'Cook County', 'state': 'Illinois', 'country': 'United States', 'country_code': 'us'}
6250 OCONOMOWOC, WISCONSIN, USA {'town': 'Oconomowoc', 'county': 'Waukesha County', 'state': 'Wisconsin', 'postcode': '53066', 'country': 'United States', 'country_code': 'us'}
6252 WILLISTON, NORTH DAKOTA, USA {'city': 'Williston', 'county': 'Williams County', 'state': 'North Dakota', 'postcode': '58801', 'country': 'United States', 'country_code': 'us'}
6253 NEW BEDFORD, MASSACHUSETTS, USA {'city': 'New Bedford', 'county': 'Bristol County', 'state': 'Massachusetts', 'postcode': '02740', 'country': 'United States', 'country_code': 'us'}
6256 CORONA DEL MAR, CALIFORNIA, UNITED STATES  {'suburb': 'Corona del Mar', 'city': 'Newport Beach', 'county': 'Orange County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
6257 SECAUCUS, NJ, NEW JERSEY, UNITED STATES {'town': 'Secauc

In [12]:
print(len(errores), errores)

867 ['PREFER NOT TO ANSWER, MARYLAND, USA', 'NYC REMOTELY, NEW YORK, US', 'PREFER NOT TO ANSWER, CALIFORNIA, USA', 'DETROIT METRO AREA, MICHIGAN, UNITED STATES', 'SMALL CITY, REMOTE, NATIONAL COMPANY, FLORIDA, UNITED STATES', 'GORHAMPORTLAND, MAINE, US', 'WESTCHESTERNYC, NEW YORK, US', 'SAN JOSE, CALIFORNIA, US', 'TOO IDENTIFIABLE, GEORGIA, USA', 'CHICAGO AREA MOSTLY, BUT ALSO THE US AND CANADA, ILLINOIS, UNITED STATES', 'GREATER BOSTON AREA, MASSACHUSETTS, USA', 'CHICAGO REMOTE , ILLINOIS, UNITED STATES', 'HUNTINGTON REMOTE, HQ IS IN CHARLESTON, WEST VIRGINIA, USA', 'TWIN CITIES METRO AREA, MINNESOTA, UNITED STATES', 'DFW AREA, TEXAS, UNITED STATES', 'FULLY REMOTE GREATER BOSTON, MASSACHUSETTS, USA', 'PREFER NOT TO SAY MONTANA IS SMALL, MONTANA, UNITED STATES', 'PANHANDLE REGION, FLORIDA, US', 'I WORK REMOTELY, NEW YORK, USA', 'REMOTE HOME, NEW YORK, UNITED STATES', 'WORK FROM HOME , MASSACHUSETTS, UNITED STATES', 'VERYSMALL TOWN, IOWA, US', 'TWIN CITIES SUBURBS , MINNESOTA, UNITED ST

Como se puede observar, hay 867 casos que presenatron algún error. En muchos de esos casis parece haber errores de ortografía, palabaras que no se deberían utilizar, o datos como ciudad faltantes. Primero nos vamos a ocupar de estos caso de error y después revisaremos la información encontrada con geocode. 

La primera aproximación que haremos, es únicamente buscar la ciudad que las personas pusieron. Esto reduciria los casos de error (cuando la gente puso el pais mal. El problema de esto, radica en que el sistema puede asumir una ciudad que está en otro pais.

Por otro lado, no se porque cada vez que se llama la función, sale que no se detectan algunas opciones que después se detectan si se vuelve a correr. Por lo tanto se hace un while que recorre todo hasta que pase directo sin encontrar coincidencias en la ciudad

In [13]:
contador = 1
while contador > 0:
    contador = 0
    for c in errores:
        try:
            lugares[c] = geolocator.geocode(c.replace(".", "").split(",")[0], exactly_one=True,language="en", namedetails=True, addressdetails=True).raw["address"]
            errores.remove(c)
            contador += 1
            print(contador, c, lugares[c])
        except:
            continue

1 DETROIT METRO AREA, MICHIGAN, UNITED STATES {'road': 'Detroit Street', 'quarter': 'Pinagkaisahan', 'suburb': 'Scout Area', 'city_district': '4th District', 'city': 'Quezon City', 'state_district': 'Eastern Manila District', 'region': 'Metro Manila', 'postcode': '1111', 'country': 'Philippines', 'country_code': 'ph'}
2 SAN JOSE, CALIFORNIA, US {'city': 'San Jose', 'county': 'Santa Clara County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
3 CHICAGO REMOTE , ILLINOIS, UNITED STATES {'building': 'Remote Parking ATS Station (Closed)', 'road': 'Coffey Road', 'suburb': "O'Hare", 'city': 'Chicago', 'municipality': 'Jefferson Township', 'county': 'Cook County', 'state': 'Illinois', 'postcode': '60666', 'country': 'United States', 'country_code': 'us'}
4 REMOTE HOME, NEW YORK, UNITED STATES {'road': 'Remote Street', 'residential': 'Modern Manor Mobile Home Park', 'municipality': 'Scott Township', 'county': 'Johnson County', 'state': 'Iowa', 'postcode': '52240', 'c

41 ATLANTA METRO AREA, GEORGIA, US {'road': 'Atlanta', 'suburb': 'Port Area', 'city_district': 'Fifth District', 'city': 'Manila', 'state_district': 'Capital District', 'region': 'Metro Manila', 'postcode': '1018', 'country': 'Philippines', 'country_code': 'ph'}
42 POOLER, GA, GEORGIA, UNITED STATESP {'town': 'Pooler', 'county': 'Chatham County', 'state': 'Georgia', 'postcode': '31322', 'country': 'United States', 'country_code': 'us'}
43 BELMONT, CA SF BAY AREA, CALIFORNIA, USA {'village': 'Belmont', 'municipality': 'Molsheim', 'county': 'Bas-Rhin', 'state': 'Grand Est', 'region': 'Metropolitan France', 'postcode': '67130', 'country': 'France', 'country_code': 'fr'}
44 REMOTE, NEW JERSEY, UNITED STATTES {'hamlet': 'Remote', 'county': 'Coos County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
45 CHATTANOOGA, GEORGIA, TENNESSEE, UNITED STATE {'city': 'Chattanooga', 'county': 'Hamilton County', 'state': 'Tennessee', 'postcode': '818', 'country': 'United States', 

77 COLORADO SPRINGS, COLORADO, US OF A {'city': 'Colorado Springs', 'county': 'El Paso County', 'state': 'Colorado', 'postcode': '80903', 'country': 'United States', 'country_code': 'us'}
78 BEAVERTON, OR PORTLAND SUBURB, OREGON, USA {'city': 'Beaverton', 'county': 'Washington County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
79 MONROE, CONNECTICUT, UNITES STATES {'county': 'Monroe County', 'state': 'Illinois', 'country': 'United States', 'country_code': 'us'}
80 PROVIDENCE, MASSACHUSETTS, RHODE ISLAND, UNITED STATES {'city': 'Providence', 'county': 'Providence County', 'state': 'Rhode Island', 'country': 'United States', 'country_code': 'us'}
81 ROSSLYN, DISTRICT OF COLUMBIA, VIRGINIA, UNITED STATES {'railway': 'Rosslyn', 'house_number': '1850', 'road': 'North Moore Street', 'suburb': 'Rosslyn', 'city': 'Arlington', 'county': 'Arlington County', 'state': 'Virginia', 'postcode': '22209', 'country': 'United States', 'country_code': 'us'}
82 PORTLAND, OREGON, 

115 NY SUBURB, NEW JERSEY, UNITED STATES {'tourism': 'Przykopa suburb', 'road': 'Przykopa', 'municipality': 'Cieszyn', 'county': 'Cieszyn County', 'state': 'Silesian Voivodeship', 'postcode': '43-400', 'country': 'Poland', 'country_code': 'pl'}
116 REMOTE, NORTH CAROLINA, UNITED STATES  {'hamlet': 'Remote', 'county': 'Coos County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
117 LOS ANGELES , ARIZONA, CALIFORNIA, NEVADA, TEXAS, USA {'city': 'Los Angeles', 'county': 'Los Angeles County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
118 SAN FRANSISCO, CALIFORNIA, USA {'hamlet': 'San Fransisco', 'village': 'Tilipulo', 'city': 'Latacunga', 'county': 'Latacunga', 'state': 'Cotopaxi', 'postcode': 'EC050101', 'country': 'Ecuador', 'country_code': 'ec'}
119 RURAL AREA, NEW YORK, USA {'landuse': 'Rural area', 'hamlet': 'El Ceibal', 'town': 'Rocafuerte', 'municipality': 'Parroquia Rocafuerte', 'county': 'Rocafuerte', 'state': 'Manabí', 'countr

155 PITTSBURGH, PENNSYLVANIA, I EARN COMMISSION ON SALES IF I MEET QUOTA, IM GUARANTEED ANOTHER K MIN LAST YEAR I EARNED AN ADDITIONAL K ITS NOT UNCOMMON FOR PEOPLE IN MY SPACE TO EARN K AFTER COMMISSION  {'city': 'Pittsburgh', 'county': 'Allegheny County', 'state': 'Pennsylvania', 'country': 'United States', 'country_code': 'us'}
156 DAVENPORT AREA, IOWA, UNITED STATES {'natural': 'Davenport Point', 'city': 'Area C (South Skeena)', 'county': 'North Coast Regional District', 'state': 'British Columbia', 'country': 'Canada', 'country_code': 'ca'}
157 SEATLE, WA, WASHINGTON, UNITED STATES {'road': 'Seatle', 'municipality': 'Ambato', 'county': 'Ambato', 'state': 'Tungurahua', 'postcode': '18050', 'country': 'Ecuador', 'country_code': 'ec'}
158 SEATTLE, WASHINGTON, UNITEDSTATES {'city': 'Seattle', 'county': 'King County', 'state': 'Washington', 'country': 'United States', 'country_code': 'us'}
159 WASHINGTON, DC IM REMOTE, BUT OFFICE IS IN DC, DISTRICT OF COLUMBIA, UNITED STATES {'city': '

...

234 MCLEAN, VIRGINIA, UNITES STATES {'county': 'McLean County', 'state': 'Illinois', 'country': 'United States', 'country_code': 'us'}
235 VARIES, NEW YORK, USA {'isolated_dwelling': 'Les Varies', 'village': 'Trébédan', 'municipality': 'Dinan', 'county': "Côtes-d'Armor", 'state': 'Brittany', 'region': 'Metropolitan France', 'postcode': '22980', 'country': 'France', 'country_code': 'fr'}
236 TWIN CITIES AREA, MINNESOTA, US {'building': 'Castel Area Health Centre', 'road': 'Constantin Lane', 'suburb': 'Castel', 'city': 'Vacoas-Phoenix', 'county': 'Vacoas-Phoenix', 'state': 'Plaines Wilhems', 'postcode': '74232', 'country': 'Mauritius', 'country_code': 'mu'}
237 SAN JOSE, CA, CALIFORNIA, Y {'city': 'San Jose', 'county': 'Santa Clara County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
238 PLEASANT PRARIE, WISCONSIN, USA  {'amenity': 'Pleasant Hill School', 'road': 'Prarie Lane', 'county': 'McCurtain County', 'state': 'Oklahoma', 'country': 'United States', 'co

275 SAN FRANCISCO, BUT COMPANY IS BOSTON BASED, CALIFORNIA, US {'city': 'San Francisco', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
276 EUGENE, OREGON, UNITED STATE {'city': 'Eugene', 'county': 'Lane County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
277 MINNEAPOLIS, MINNESOTA, UNITED SATES {'city': 'Minneapolis', 'county': 'Hennepin County', 'state': 'Minnesota', 'postcode': '55488', 'country': 'United States', 'country_code': 'us'}
278 ALL, TRAVEL, NORTH CAROLINA, USA {'county': 'Allen County', 'state': 'Ohio', 'country': 'United States', 'country_code': 'us'}
279 STL, MISSISSIPPI, MISSOURI, USA {'state': 'Southland', 'country': 'New Zealand', 'country_code': 'nz'}
280 LOS ANGELES, CALIFORNIA, MARYLAND, UNITED STATES {'city': 'Los Angeles', 'county': 'Los Angeles County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
281 TERRE HAUTE, INDIANA, UNITED STATES IS AMERICA {'city': 'Terre Haute', 'county': 

32 CHICAGO SUBURBS, INDIANA, USA {'suburb': 'Chicago', 'town': 'Paarl', 'city': 'Drakenstein Local Municipality', 'county': 'Cape Winelands District Municipality', 'state': 'Western Cape', 'postcode': '7646', 'country': 'South Africa', 'country_code': 'za'}
33 ALEXANDRIA, VA, DISTRICT OF COLUMBIA, US {'city': 'Alexandria', 'state': 'Alexandria Governorate', 'postcode': '21519', 'country': 'Egypt', 'country_code': 'eg'}
34 PORTLAND, CALIFORNIA, OREGON, UNITED STATES {'city': 'Portland', 'county': 'Multnomah County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
35 PORTLAND, OREGON, CURRENTLY FINANCE {'city': 'Portland', 'county': 'Multnomah County', 'state': 'Oregon', 'country': 'United States', 'country_code': 'us'}
36 REMOVE, VIRGINIA, USA {'waterway': 'East Fork Point Remove Site Three Dam', 'county': 'Conway County', 'state': 'Arkansas', 'country': 'United States', 'country_code': 'us'}
37 WASHINGTON DC, DISTRICT OF COLUMBIA, UXZ {'city': 'Washington', 'state'

72 EAST ROCKAWAY LONG ISLAND, NEW YORK, USA {'shop': 'Hearing Center of Long Island', 'house_number': '46', 'road': 'Rockaway Avenue', 'town': 'Valley Stream', 'municipality': 'Town of Hempstead', 'county': 'Nassau County', 'state': 'New York', 'postcode': '11580', 'country': 'United States', 'country_code': 'us'}
73 BARDSTOWN, KY  STORE IS LOCATED IN FRANKFORT, KENTUCKY, USA {'town': 'Bardstown', 'city': 'Bardstown', 'county': 'Nelson County', 'state': 'Kentucky', 'postcode': '40004', 'country': 'United States', 'country_code': 'us'}
74 SAN FRANSICO, CALIFORNIA, UNITED STATES {'locality': 'San Fransico (6)', 'town': 'Contadero', 'county': 'Contadero', 'state_district': 'Ex-Providencia de Obando', 'state': 'Nariño', 'country': 'Colombia', 'country_code': 'co'}
75 BLANK, MISSISSIPPI, I WAS BROUGHT IN ON THIS SALARY TO HELP WITH THE EHR AND VERY QUICKLY WAS PROMOTED TO CURRENT POSITION BUT COMPENSATION WAS NOT ALTERED  {'hamlet': 'Blank', 'county': 'Frederick County', 'state': 'Maryland'

109 NA, ALABAMA, NA {'country': 'Namibia', 'country_code': 'na'}
110 NEW YORK CITY , NEW YORK, VIRGINIA, USA  {'city': 'New York', 'state': 'New York', 'country': 'United States', 'country_code': 'us'}
111 DECLINE, NEW JERSEY, UNITED STATES {'industrial': 'Decline 4 South', 'suburb': 'Moses Kotane Ward 7', 'city': 'Moses Kotane Local Municipality', 'county': 'Bojanala Platinum District Municipality', 'state': 'North West', 'country': 'South Africa', 'country_code': 'za'}
112 RALEIGH, CALIFORNIA, ILLINOIS, MASSACHUSETTS, NORTH CAROLINA, SOUTH CAROLINA, VIRGINIA, USA  {'city': 'Raleigh', 'county': 'Wake County', 'state': 'North Carolina', 'country': 'United States', 'country_code': 'us'}
113 CHICAGO, ILLINOIS, UNITED STATE OF AMERICA {'city': 'Chicago', 'county': 'Cook County', 'state': 'Illinois', 'country': 'United States', 'country_code': 'us'}
114 BOISE, IDAHO, UNITED STATES OF AMERICAN {'city': 'Boise', 'county': 'Ada County', 'state': 'Idaho', 'postcode': '83701', 'country': 'Unite

Como se puede observar, se disminuye el caso de errores. Por lo tanto, vamos a seguir con el siguiente nivel (intentando únicamente los States de las entradas que mantienen su error).

In [14]:
contador = 1
while contador > 0:
    contador = 0
    for c in errores:
        try:
            lugares[c] = geolocator.geocode(c.replace(".", "").split(",")[1], exactly_one=True,language="en", namedetails=True, addressdetails=True).raw["address"]
            errores.remove(c)
            contador += 1
            print(contador, c, lugares[c])
        except:
            continue

1 PREFER NOT TO ANSWER, MARYLAND, USA {'state': 'Maryland', 'country': 'United States', 'country_code': 'us'}
2 PREFER NOT TO ANSWER, CALIFORNIA, USA {'state': 'California', 'country': 'United States', 'country_code': 'us'}
3 WESTCHESTERNYC, NEW YORK, US {'city': 'New York', 'state': 'New York', 'country': 'United States', 'country_code': 'us'}
4 GREATER BOSTON AREA, MASSACHUSETTS, USA {'state': 'Massachusetts', 'country': 'United States', 'country_code': 'us'}
5 TWIN CITIES METRO AREA, MINNESOTA, UNITED STATES {'state': 'Minnesota', 'country': 'United States', 'country_code': 'us'}
6 FULLY REMOTE GREATER BOSTON, MASSACHUSETTS, USA {'state': 'Massachusetts', 'country': 'United States', 'country_code': 'us'}
7 PANHANDLE REGION, FLORIDA, US {'state': 'Florida', 'country': 'United States', 'country_code': 'us'}
8 WORK FROM HOME , MASSACHUSETTS, UNITED STATES {'state': 'Massachusetts', 'country': 'United States', 'country_code': 'us'}
9 LAS VEGAS TECHINICALLY REMOTE, NEVADA, UNITED STATES 

68 PREFER NOT TO ANSWER, ILLINOIS, USA {'state': 'Illinois', 'country': 'United States', 'country_code': 'us'}
69 SMALL CITYACTUAL CITY WOULD GIVE AWAY THE COMPANY, CALIFORNIA, USA {'state': 'California', 'country': 'United States', 'country_code': 'us'}
70 I WORK REMOTELY BUT IN NONCOVID TIMES I WORK AT CLIENT SITES THROUGHOUT THE US, NEW YORK, UNITED STATES {'city': 'New York', 'state': 'New York', 'country': 'United States', 'country_code': 'us'}
71 ATLANTADECATUR, GEORGIA, UNITED STATES {'country': 'Georgia', 'country_code': 'ge'}
72 ONE OF THE LARGEST IN THE STATE, WISCONSIN, USA {'state': 'Wisconsin', 'country': 'United States', 'country_code': 'us'}
73 OUTSIDE THE TWIN CITIES, MINNESOTA, USA {'state': 'Minnesota', 'country': 'United States', 'country_code': 'us'}
74 NOT SAYING , IOWA, UNITED STATES  {'state': 'Iowa', 'country': 'United States', 'country_code': 'us'}
75 I WORK REMOTELY WITH CLIENTS ACROSS THE COUNTRY, INDIANA, UNITED STATES {'state': 'Indiana', 'country': 'United

135 SEATLLE, WASHINGTON, USA {'city': 'Washington', 'state': 'District of Columbia', 'country': 'United States', 'country_code': 'us'}
136 VERNON  REMOTE POSITION THOUGH , INDIANA, USA {'state': 'Indiana', 'country': 'United States', 'country_code': 'us'}
137 MINNEAPOLISST PAUL, MINNESOTA, US {'state': 'Minnesota', 'country': 'United States', 'country_code': 'us'}
138 SEATTLE COMPANY IS IN REDMOND, WASHINGTON, USA {'city': 'Washington', 'state': 'District of Columbia', 'country': 'United States', 'country_code': 'us'}
139 PREFER NOT TO ANSWER, LOUISIANA, UNITED STATES {'state': 'Louisiana', 'country': 'United States', 'country_code': 'us'}
140 GREATER SEATTLE AREA, WASHINGTON, UNITED STATES  {'city': 'Washington', 'state': 'District of Columbia', 'country': 'United States', 'country_code': 'us'}
141 DMV METRO AREA, MARYLAND, VIRGINIA, USA {'state': 'Maryland', 'country': 'United States', 'country_code': 'us'}
142 CANT SAY, MISSISSIPPI, UNITED STATES {'state': 'Mississippi', 'country': 

204 OAKLAND AND HAYWARD, CALIFORNIA, UNITED STATES {'state': 'California', 'country': 'United States', 'country_code': 'us'}
205 PREFER NOT TO SAY, LOUISIANA, USA {'state': 'Louisiana', 'country': 'United States', 'country_code': 'us'}
206 MILWAUKEE REMOTE, WISCONSIN, USA {'state': 'Wisconsin', 'country': 'United States', 'country_code': 'us'}
207 SF BAY AREA REMOTE, CALIFORNIA, US {'state': 'California', 'country': 'United States', 'country_code': 'us'}
208 MILWAUKEEREMOTE, WISCONSIN, UNITED STATES {'state': 'Wisconsin', 'country': 'United States', 'country_code': 'us'}
209 RALEIGH DURHAM AREA , NORTH CAROLINA, UNITED STATES {'state': 'North Carolina', 'country': 'United States', 'country_code': 'us'}
1 NYC REMOTELY, NEW YORK, US {'city': 'New York', 'state': 'New York', 'country': 'United States', 'country_code': 'us'}
2 TOO IDENTIFIABLE, GEORGIA, USA {'country': 'Georgia', 'country_code': 'ge'}
3 HUNTINGTON REMOTE, HQ IS IN CHARLESTON, WEST VIRGINIA, USA {'locality': 'Charleston', '

60 BRYANCOLLEGE STATION, TEXAS, US {'state': 'Texas', 'country': 'United States', 'country_code': 'us'}
61 WINSTONSALEM, NORTH CAROLINA, USA {'state': 'North Carolina', 'country': 'United States', 'country_code': 'us'}
62 LOUISVILLE REMOTE, KENTUCKY, USA {'state': 'Kentucky', 'country': 'United States', 'country_code': 'us'}
63 DECLINE TO ANSWER, UTAH, UNITED STATES {'state': 'Utah', 'country': 'United States', 'country_code': 'us'}
64 FULLY REMOTE, WASHINGTON, UNITED STATES {'city': 'Washington', 'state': 'District of Columbia', 'country': 'United States', 'country_code': 'us'}
65 OGDEN  BUT I WORK REMOTE COMPANY IS BASED IN MD, UTAH, UNITED STATES {'state': 'Utah', 'country': 'United States', 'country_code': 'us'}
66 GREATER MINNEAPOLIS METRO AREA, MINNESOTA, UNITED STATES {'state': 'Minnesota', 'country': 'United States', 'country_code': 'us'}
67 NY METRO AREA BUT NOT NYC, NEW YORK, USA {'city': 'New York', 'state': 'New York', 'country': 'United States', 'country_code': 'us'}
68 RE

23 LYNNFIELD BOSTON METRO AREA, MASSACHUSETTS, UNITED STATES {'state': 'Massachusetts', 'country': 'United States', 'country_code': 'us'}
24 PREFER NOT TO SAY, HAWAII, USA {'state': 'Hawaii', 'country': 'United States', 'country_code': 'us'}
25 GROVE CITY  REMOTE, OHIO, USA {'state': 'Ohio', 'country': 'United States', 'country_code': 'us'}
26 ALLENPLANO, TEXAS, UNITED STATES {'state': 'Texas', 'country': 'United States', 'country_code': 'us'}
27 REMOTELY  STAMFORD HQ, CONNECTICUT, USA {'state': 'Connecticut', 'country': 'United States', 'country_code': 'us'}
28 ANAHEIMSANTA ANAIRVINE METRO REGION, CALIFORNIA, UNITED STATES {'state': 'California', 'country': 'United States', 'country_code': 'us'}
29 I WORK IN A RURAL AREA AND CANNOT ANSWER THIS QUESTION WHILE MAINTAINING ANONYMITY, MAINE, UNITED STATES {'state': 'Maine', 'country': 'United States', 'country_code': 'us'}
30 THE OFFICE IS IN SCHAUMBURG I WORK REMOTELY, ILLINOIS, UNITED STATES {'state': 'Illinois', 'country': 'United Stat

Por último, haremos lo mismo pero con el pais.

In [15]:
contador = 1
while contador > 0:
    contador = 0
    for c in errores:
        try:
            lugares[c] = geolocator.geocode(c.replace(".", "").split(",")[-1], exactly_one=True,language="en", namedetails=True, addressdetails=True).raw["address"]
            errores.remove(c)
            contador += 1
            print(contador, c, lugares[c])
        except:
            continue

1 CHICAGO AREA MOSTLY, BUT ALSO THE US AND CANADA, ILLINOIS, UNITED STATES {'country': 'United States', 'country_code': 'us'}
2 JOB IS BASED IN DC, BUT I GO TO WHERE DISASTERS ARE , DISTRICT OF COLUMBIA, USA {'country': 'United States', 'country_code': 'us'}
3 COMPANY IS IN NYC, IM REMOTE IN BOSTON, NEW YORK, UNITED STATES {'country': 'United States', 'country_code': 'us'}
4 STILLWATER WI BASED COMPANY, OFFICE IN MN, MINNESOTA, US {'country': 'United States', 'country_code': 'us'}
5 CORP HQ IS IN CHARLESTON, SC I WFH, MINNESOTA, US {'country': 'United States', 'country_code': 'us'}
6 MONTCLAIR NOT ACTUAL CITY, BUT SAME REGION FOR ANONYMITY, NEW JERSEY, USA {'country': 'United States', 'country_code': 'us'}
7 TELECOMMUTE FROM ANAHEIM, CA EMPLOYER OFFICE IN RANCHO CUCAMONGA, CA, CALIFORNIA, UNITED STATES {'country': 'United States', 'country_code': 'us'}
8 OFFICE WAS IN CHICAGO, NOW WFH IN WI, ILLINOIS, WISCONSIN, USA {'country': 'United States', 'country_code': 'us'}
9 I WORK REMOTELY, 

In [17]:
errores

[]

Como se puede observar, todas las ubicaciones ya tienen al menos un objeto referenciado. Es importante aclarar que claramente esto puede tener algunos errores, pero de igual manera es una buena forma de estandarización. 

Ahora, se crea una nueva variable Pais, con los paises enocntrados en el método. Pude evidenciar que en dos casos no se encontró ningún pais, aún así ambos casos son en Alaska y se refiere a lugares en USA por lo tanto, cree una función que retorna el pais y si no encuentra entonces asigna "United States"

In [29]:
def extaer_pais(x):
    try:
        return(x["country"])
    except:
        return("United States")
    
df["Pais"] = df["Lugar"].apply(lambda x: extaer_pais(lugares[x]))

Por otro lado, hacemos lo mismo con la ciudad. Este caso es más delicado porque muchos sitios no se encontró la ciudad. por lo tanto (y después de muchas iteraciones en el proceso) se llegó a la siguiente conclusión:

Primero se eliminan todos los valores que en lugar tienen NAN (esto se ha debido hacer antes pero se me olvidó y en este punto se demora mucho corriendo todo el código)

después, se extrae los que tienen "city"
Tercero, se extrae los que tienen "town"
porteriormente, los que tiene. "place"
Quinto, los que tienen "suburb"
sexto, los que tiene "village"
septimo, los que tienen "locality"
octavo, los que tienen "county". En este punto nos vamos extendiendo a territorios más grandes.

Despues de hacer esto se mantiene la incógnita en 421 registros. Es decir, registros que no tienen ciudad, solo tienen estado o pais. Para estos casos vamos a hacer un proceso un poco diferente. Si tiene estado, entonces vamos a coger el punto centralk del estado y si solo tiene pais, el punto central del pais. Para eso creamos la función ubicacion_central

9. Se hace todo el proceso descrito anteriormente. 
10. Si persiste problema se hizo manualmente, fueron solo 3 casos.

In [35]:
df = df[df["Lugar"] != "NAN"]

In [89]:
def ubicacion_central(lugar):
    loc = geolocator.geocode(lugar, exactly_one=True,language="en", addressdetails=True,).raw
    location = geolocator.reverse(loc["lat"]+","+loc["lon"])
    return(location.raw["address"])

In [94]:
def extaer_ciudad(x):
    try:
        return(x["city"])
    except:
        try:
            return(x["town"])
        except:
            try:
                return(x["place"])
            except:
                try:
                    return(x["suburb"])
                except:
                    try:
                        return(x["village"])
                    except:
                        try:
                            return(x["locality"])
                        except:
                            try:
                                return(x["county"])
                            except:
                                try:
                                    return(ubicacion_central(x["state"])["county"])
                                except:
                                    try:
                                        return(ubicacion_central(x["country"])["county"])
                                    except:
                                        return("")



In [96]:
df["Ciudad"] = df["Lugar"].apply(lambda x: extaer_ciudad(lugares[x]))

<ipython-input-96-6b7264b32797>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ciudad"] = df["Lugar"].apply(lambda x: extaer_ciudad(lugares[x]))


In [98]:
df[df["Ciudad"] == ""]

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,...,Years of experience in field,Highest level of education completed,Gender,Race,salario_anual,compensaciones,Total_ingresos,Lugar,Pais,Ciudad
2006,4/27/2021 11:36:17,35-44,Nonprofits,Program administrator,My official title is program assistant but I a...,"64,000",1500.0,USD,NaN,NaN,...,11 - 20 years,College degree,Woman,White,250721280.0,58762800.0,309484080.0,"RURAL NORTHERN, MINNESOTA, USA",Tanzania,
16808,4/28/2021 18:34:04,25-34,Engineering or Manufacturing,Field Geologist,NaN,"61,000",0.0,USD,NaN,NaN,...,2 - 4 years,Master's degree,Woman,Another option not listed here or prefer not t...,238968720.0,0.0,238968720.0,"SOUTH, FLORIDA, GEORGIA, SOUTH CAROLINA, USA",South Korea,
26562,7/20/2021 14:58:53,under 18,"Accounting, Banking & Finance",na,na,00,0.0,USD,na,na,...,1 year or less,High School,Man,Asian or Asian American,0.0,0.0,0.0,"NA, ALABAMA, NA",Namibia,


In [99]:
df.loc[2006, "Pais"] = "United States"
df.loc[2006, "Ciudad"] = "Saint Paul"
df.loc[16808, "Pais"] = "United States"
df.loc[16808, "Ciudad"] = "Georgia"
df.loc[26562, "Pais"] = "United States"
df.loc[26562, "Ciudad"] = "Montgomery"

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [101]:
df.to_csv("Corregido.csv")